    Las distintas solicitudes de proyectos a financiar dentro de la comunidad de castilla y leon, junto con sus objetivos y en cuantos se ha acabado invirtiendo.

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.express as px
from pyspark.sql import functions as f
import plotly.graph_objects as go

In [ ]:
from pyspark.sql import SparkSession

my_spark = SparkSession \
    .builder \
    .appName("PySparkShell") \
    .config("spark.mongodb.input.uri", "mongodb://127.0.0.1/castillaYLeon.BalancePlataformaFinanciera") \
    .config("spark.mongodb.output.uri", "mongodb://127.0.0.1/castillaYLeon.BalancePlataformaFinanciera") \
    .getOrCreate()

In [ ]:
df = spark.read.format("mongo").load()
df = df.drop("_id")

In [ ]:
df= df.withColumnRenamed("Objetivo - Inversi�n","Objetivo-Inversion")

In [ ]:
df.show()

In [ ]:
df.take(1)

In [ ]:
df.select("Mes").distinct().show()

In [ ]:
#df = df.withColumn("Mes",f.to_date(df.Mes,'MM').alias('dt'))

In [ ]:
df.select("Anualidad").distinct().orderBy("Anualidad").show()

In [ ]:
df.select("Mes").take(1)[0][0]

In [ ]:
#dfPrueba = df.toPandas()

                Convierte los meses en español a numero en una columna

In [ ]:
from pyspark.sql.functions import regexp_replace

In [ ]:
def cambio(column):
    col=regexp_replace(column,'Enero','1')
    col=regexp_replace(col,'Febrero','2')
    col=regexp_replace(col,'Marzo','3')
    col=regexp_replace(col,'Abril','4')
    col=regexp_replace(col,'Mayo','5')
    col=regexp_replace(col,'Junio','6')
    col=regexp_replace(col,'Julio','7')
    col=regexp_replace(col,'Agosto','8')
    col=regexp_replace(col,'Septiembre','9')
    col=regexp_replace(col,'Octubre','10')
    col=regexp_replace(col,'Noviembre','11')
    col=regexp_replace(col,'Diciembre','12')
    return col

In [ ]:
columnaNumeros= cambio(df.Mes)

In [ ]:
df = df.withColumn("numeroMes",columnaNumeros)

In [ ]:
df.select("numeroMes").collect()

In [ ]:
df=df.drop("field30")
df=df.drop("field31")

In [ ]:
df = df.withColumn("numeroMes",df.numeroMes.cast("Integer"))
df = df.withColumn("Sector Agroalimentario",df["Sector Agroalimentario"].cast("Integer"))
#df = df.withColumn("Sector Auxiliar Automocion",df["Sector Auxiliar Automocion"].cast("Integer"))
df = df.withColumn("Sector Biotecnologia",df["Sector Biotecnologia"].cast("Integer"))
df = df.withColumn("Sector Comercio",df["Sector Comercio"].cast("Integer"))
df = df.withColumn("Sector Energia",df["Sector Energia"].cast("Integer"))
df = df.withColumn("Sector Fabricacion equipos",df["Sector Fabricacion equipos"].cast("Integer"))
df = df.withColumn("Sector Hosteleria",df["Sector Hosteleria"].cast("Integer"))
df = df.withColumn("Sector Otros",df["Sector Otros"].cast("Integer"))
df = df.withColumn("Sector Quimico Farmaceutico",df["Sector Quimico Farmaceutico"].cast("Integer"))
df = df.withColumn("Sector Reciclado",df["Sector Reciclado"].cast("Integer"))
df = df.withColumn("Sector Servicios",df["Sector Servicios"].cast("Integer"))
df = df.withColumn("Sector Textil",df["Sector Textil"].cast("Integer"))
df = df.withColumn("Sector Transformados madera",df["Sector Transformados madera"].cast("Integer"))
df = df.withColumn("Sector Transformados metal",df["Sector Transformados metal"].cast("Integer"))
df = df.withColumn("Sector Transformados plastico",df["Sector Transformados plastico"].cast("Integer"))
df = df.withColumn("Sector Transformados vidrio",df["Sector Transformados vidrio"].cast("Integer"))
df = df.withColumn("Sector Transporte",df["Sector Transporte"].cast("Integer"))

In [ ]:
df.printSchema()

In [ ]:
df = df.sort("Anualidad")

In [ ]:
dfPrueba = df.toPandas()

In [ ]:
dfPrueba["Tipo de Solicitud"].unique()

In [ ]:
#aux1.sort_values(by=['numeroMes'],ascending=True).head(5)

In [ ]:
figFinancieraAnualBarras = go.Figure(layout_yaxis_range=[0,100])

# Add traces, one for each slider step
for years in dfPrueba["Anualidad"].unique(): #recorro un bucle con cada año
    aux = dfPrueba[dfPrueba["Anualidad"]==years] #Creo un aux que sera el df con el año que necesito
    aux1 = aux[aux["Tipo de Solicitud"] == "Solicitudes presentadas"]
    #aux2 = aux[aux["Tipo de Solicitud"] == "Solicitudes tramitadas - financiadas"]
    #aux3 = aux[aux["Tipo de Solicitud"] == "Solicitudes tramitadas - asesoradas"]
    #aux4 = aux[aux["Tipo de Solicitud"] == "Solicitudes tramitadas - otras"]
    aux1 = aux1.sort_values(by=['numeroMes'])
    #aux1= aux1.sort(aux1.numeroMes)
    figFinancieraAnualBarras.add_bar(
        visible=False, #las ocultamos
        name="Sector Servicios",
        x=aux1.Mes,
        y=aux1["Sector Servicios"]
    )
    figFinancieraAnualBarras.add_bar(
        visible=False, # las ocultamos
        name="Sector Textil",
        x=aux1.Mes,
        y=aux["Sector Textil"]
    )
    
    figFinancieraAnualBarras.add_bar(
        visible=False, #las ocultamos
        name="Sector Agroalimentario",
        x=aux1.Mes,
        y=aux1["Sector Agroalimentario"]
    )
    figFinancieraAnualBarras.add_bar(
        visible=False, # las ocultamos
        name="Sector Auxiliar Automocion",
        x=aux1.Mes,
        y=aux["Sector Auxiliar Automocion"]
    )
    figFinancieraAnualBarras.add_bar(
        visible=False, #las ocultamos
        name="Sector Biotecnologia",
        x=aux1.Mes,
        y=aux1["Sector Biotecnologia"]
    )
    figFinancieraAnualBarras.add_bar(
        visible=False, # las ocultamos
        name="Sector Comercio",
        x=aux1.Mes,
        y=aux["Sector Comercio"]
    )
    figFinancieraAnualBarras.add_bar(
        visible=False, #las ocultamos
        name="Sector Energia",
        x=aux1.Mes,
        y=aux1["Sector Energia"]
    )
    figFinancieraAnualBarras.add_bar(
        visible=False, # las ocultamos
        name="Sector Fabricacion equipos",
        x=aux1.Mes,
        y=aux["Sector Fabricacion equipos"]
    )
    figFinancieraAnualBarras.add_bar(
        visible=False, #las ocultamos
        name="Sector Hosteleria",
        x=aux1.Mes,
        y=aux1["Sector Hosteleria"]
    )
    figFinancieraAnualBarras.add_bar(
        visible=False, # las ocultamos
        name="Sector Otros",
        x=aux1.Mes,
        y=aux["Sector Otros"]
    )
    figFinancieraAnualBarras.add_bar(
        visible=False, #las ocultamos
        name="Sector Quimico Farmaceutico",
        x=aux1.Mes,
        y=aux1["Sector Quimico Farmaceutico"]
    )
    figFinancieraAnualBarras.add_bar(
        visible=False, # las ocultamos
        name="Sector Reciclado",
        x=aux1.Mes,
        y=aux["Sector Reciclado"]
    )
    figFinancieraAnualBarras.add_bar(
        visible=False, #las ocultamos
        name="Sector Tic",
        x=aux1.Mes,
        y=aux1["Sector Tic"]
    )
    figFinancieraAnualBarras.add_bar(
        visible=False, # las ocultamos
        name="Sector Transformados madera",
        x=aux1.Mes,
        y=aux["Sector Transformados madera"]
    )
    figFinancieraAnualBarras.add_bar(
        visible=False, #las ocultamos
        name="Sector Transformados metal",
        x=aux1.Mes,
        y=aux1["Sector Transformados metal"]
    )
    figFinancieraAnualBarras.add_bar(
        visible=False, # las ocultamos
        name="Sector Transformados plastico",
        x=aux1.Mes,
        y=aux["Sector Transformados plastico"]
    )
    figFinancieraAnualBarras.add_bar(
        visible=False, # las ocultamos
        name="Sector Transformados vidrio",
        x=aux1.Mes,
        y=aux["Sector Transformados vidrio"]
    )
    figFinancieraAnualBarras.add_bar(
        visible=False, # las ocultamos
        name="Sector Transporte",
        x=aux1.Mes,
        y=aux["Sector Transporte"]
    )
        

# Hacemos que la 1º y la 2º barra de la figura sean visibles, de esta forma nada mas cargue el grafico nos los mostrara por pantalla.
figFinancieraAnualBarras.data[0].visible = True
figFinancieraAnualBarras.data[1].visible = True
figFinancieraAnualBarras.data[2].visible = True
figFinancieraAnualBarras.data[3].visible = True
figFinancieraAnualBarras.data[4].visible = True
figFinancieraAnualBarras.data[5].visible = True
figFinancieraAnualBarras.data[6].visible = True
figFinancieraAnualBarras.data[7].visible = True
figFinancieraAnualBarras.data[8].visible = True
figFinancieraAnualBarras.data[9].visible = True
figFinancieraAnualBarras.data[10].visible = True
figFinancieraAnualBarras.data[11].visible = True
figFinancieraAnualBarras.data[12].visible = True
figFinancieraAnualBarras.data[13].visible = True
figFinancieraAnualBarras.data[14].visible = True
figFinancieraAnualBarras.data[15].visible = True
figFinancieraAnualBarras.data[16].visible = True
figFinancieraAnualBarras.data[17].visible = True
# Create and add slider
steps = []
#steps, son los pasos , los cambios que hay entre cada año
# es en los steps donde hacemos visibles las barras y actualizamos la grafica
for i in range(0,len(figFinancieraAnualBarras.data),18): # recorro un bucle con todos los datos de la grafica, hago una iteracion de 2 en 2 debido a que me interesa sacar dos datos cada vez, dos barras
    step = dict(
        method="update", 
        args=[{"visible": [False] * len(figFinancieraAnualBarras.data)},  
              {"title": "Gráfica Solicitud Presentada año: " + str(dfPrueba.Anualidad.unique().tolist()[int(i/18)])},
             
             ] # layout attribute y nombre de la grafica
    )
    
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible", hacemos visible esta barra
    step["args"][0]["visible"][i+1] = True #y esta tambien pues necesitamos hacer visibles a las dos barras
    step["args"][0]["visible"][i+2] = True
    step["args"][0]["visible"][i+3] = True
    step["args"][0]["visible"][i+4] = True
    step["args"][0]["visible"][i+5] = True
    step["args"][0]["visible"][i+6] = True
    step["args"][0]["visible"][i+7] = True
    step["args"][0]["visible"][i+8] = True
    step["args"][0]["visible"][i+9] = True
    step["args"][0]["visible"][i+10] = True
    step["args"][0]["visible"][i+11] = True
    step["args"][0]["visible"][i+12] = True
    step["args"][0]["visible"][i+13] = True
    step["args"][0]["visible"][i+14] = True
    step["args"][0]["visible"][i+15] = True
    step["args"][0]["visible"][i+16] = True
    step["args"][0]["visible"][i+17] = True
    
    steps.append(step)
#sliders
sliders = [dict(
    active=dfPrueba["Anualidad"].nunique(),
    currentvalue={"prefix":"Año: "}, #el prefijo que aparece entre la grafica y justo encima del slider
    pad={"t": dfPrueba["Anualidad"].nunique()}, #el numero de steps que tendra el slider
    steps=steps,
    xanchor='left',
    y= -0.2
)]

##
#añado los sliders y estilizo un poco el grafico 
figFinancieraAnualBarras.update_layout(
    sliders=sliders,
    xaxis_title = "Periodo",
    yaxis_title = "Índice de Precios",
    legend_title="Precios",
    font=dict(
        color="black"
    ),
    title={
        'text': "Gráfica Solicitudes Presentadas Anual en Barra",
        'y': 0.9,
        'x':0.4,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    
)

figFinancieraAnualBarras.update_xaxes(title_standoff=30)
##
# Edit slider labels
figFinancieraAnualBarras['layout']['sliders'][0]['currentvalue']['prefix']='Año: '
for i, date in enumerate(dfPrueba["Anualidad"].unique().tolist(), start = 0):
#for i,date in enumerate(lista, start = 0):
    figFinancieraAnualBarras['layout']['sliders'][0]['steps'][i]['label']= date
    

figFinancieraAnualBarras.show()

# Dejo la anterior grafica de sliders igual para poder ver como las hacia antes, 
# A partir de ahora usare el metodo siguiente para cuando tenga que mostrar una gran cantidad de barras.

In [ ]:
def switch(argument):
    switcher = {
        0: "Sector Servicios",
        1: "Sector Textil",
        2: "Sector Agroalimentario",
        3: "Sector Auxiliar Automocion",
        4: "Sector Biotecnologia",
        5: "Sector Comercio",
        6: "Sector Energia",
        7: "Sector Fabricacion equipos",
        8: "Sector Hosteleria",
        9: "Sector Otros",
        10: "Sector Quimico Farmaceutico",
        11: "Sector Reciclado",
        12: "Sector Tic",
        13: "Sector Transformados madera",
        14: "Sector Transformados metal",
        15: "Sector Transformados plastico",
        16: "Sector Transformados vidrio",
        17: "Sector Transporte"
    }
    return switcher.get(argument)

In [ ]:
figFinancieraAnualBarrasTramitadasFinanciadas = go.Figure(layout_yaxis_range=[0,100])

# Add traces, one for each slider step
for years in dfPrueba["Anualidad"].unique(): #recorro un bucle con cada año
    aux = dfPrueba[dfPrueba["Anualidad"]==years] #Creo un aux que sera el df con el año que necesito
    aux1 = aux[aux["Tipo de Solicitud"] == "Solicitudes tramitadas - financiadas"]
    aux1 = aux1.sort_values(by=['numeroMes'])
    #hacer un bucle for
    
    for i in range(18):
            
        nombre = switch (i)
        columna = aux1[nombre]
        
        figFinancieraAnualBarrasTramitadasFinanciadas.add_bar(
            visible=False, #las ocultamos
            x=aux1.Mes,
            name= nombre,
            y=columna
        )
        
    
# con Este bucle anidado mostramos tantas columnas como vueltas demos de la primera grafica
for i in range (18):
    figFinancieraAnualBarrasTramitadasFinanciadas.data[i].visible = True
# Create and add slider
steps = []
#steps, son los pasos , los cambios que hay entre cada año
# es en los steps donde hacemos visibles las barras y actualizamos la grafica
for i in range(0,len(figFinancieraAnualBarrasTramitadasFinanciadas.data),18): # recorro un bucle con todos los datos de la grafica, hago una iteracion de 2 en 2 debido a que me interesa sacar dos datos cada vez, dos barras
    step = dict(
        method="update", 
        args=[{"visible": [False] * len(figFinancieraAnualBarrasTramitadasFinanciadas.data)},  
              {"title": "Gráfica Solicitudes Tramitadas Financiadas año: " + str(dfPrueba.Anualidad.unique().tolist()[int(i/18)])},
             
             ] # layout attribute y nombre de la grafica
    )
    
    for u in range(18): # Una forma optimizada y mas bonita que antes, un bucle anidado para cada barra
        step["args"][0]["visible"][i+u] = True
    
    
    steps.append(step)
#sliders
sliders = [dict(
    active=dfPrueba["Anualidad"].nunique(),
    currentvalue={"prefix":"Año: "}, #el prefijo que aparece entre la grafica y justo encima del slider
    pad={"t": dfPrueba["Anualidad"].nunique()}, #el numero de steps que tendra el slider
    steps=steps,
    xanchor='left',
    y= -0.2
)]

##
#añado los sliders y estilizo un poco el grafico 
figFinancieraAnualBarrasTramitadasFinanciadas.update_layout(
    sliders=sliders,
    xaxis_title = "Periodo",
    yaxis_title = "Índice de Precios",
    legend_title="Precios",
    font=dict(
        color="black"
    ),
    title={
        'text': "Gráfica Solicitudes Tramitadas Financiadas Anual en Barra",
        'y': 0.9,
        'x':0.4,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    
)

figFinancieraAnualBarrasTramitadasFinanciadas.update_xaxes(title_standoff=30)
##
# Edit slider labels
figFinancieraAnualBarrasTramitadasFinanciadas['layout']['sliders'][0]['currentvalue']['prefix']='Año: '
for i, date in enumerate(dfPrueba["Anualidad"].unique().tolist(), start = 0):
#for i,date in enumerate(lista, start = 0):
    figFinancieraAnualBarrasTramitadasFinanciadas['layout']['sliders'][0]['steps'][i]['label']= date
    

figFinancieraAnualBarrasTramitadasFinanciadas.show()

In [ ]:
figFinancieraAnualBarrasTramitadasAsesoradas = go.Figure(layout_yaxis_range=[0,100])

# Add traces, one for each slider step
for years in dfPrueba["Anualidad"].unique(): #recorro un bucle con cada año
    aux = dfPrueba[dfPrueba["Anualidad"]==years] #Creo un aux que sera el df con el año que necesito
    aux1 = aux[aux["Tipo de Solicitud"] == "Solicitudes tramitadas - asesoradas"]
    aux1 = aux1.sort_values(by=['numeroMes'])
    #hacer un bucle for
    
    for i in range(18):
            
        nombre = switch (i)
        columna = aux1[nombre]
        
        figFinancieraAnualBarrasTramitadasAsesoradas.add_bar(
            visible=False, #las ocultamos
            x=aux1.Mes,
            name= nombre,
            y=columna
        )
        
    
# con Este bucle anidado mostramos tantas columnas como vueltas demos de la primera grafica
for i in range (18):
    figFinancieraAnualBarrasTramitadasAsesoradas.data[i].visible = True
# Create and add slider
steps = []
#steps, son los pasos , los cambios que hay entre cada año
# es en los steps donde hacemos visibles las barras y actualizamos la grafica
for i in range(0,len(figFinancieraAnualBarrasTramitadasAsesoradas.data),18): # recorro un bucle con todos los datos de la grafica, hago una iteracion de 2 en 2 debido a que me interesa sacar dos datos cada vez, dos barras
    step = dict(
        method="update", 
        args=[{"visible": [False] * len(figFinancieraAnualBarrasTramitadasAsesoradas.data)},  
              {"title": "Gráfica Solicitudes Tramitadas Asesoradas año: " + str(dfPrueba.Anualidad.unique().tolist()[int(i/18)])},
             
             ] # layout attribute y nombre de la grafica
    )
    
    for u in range(18): # Una forma optimizada y mas bonita que antes, un bucle anidado para cada barra
        step["args"][0]["visible"][i+u] = True
    
    
    steps.append(step)
#sliders
sliders = [dict(
    active=dfPrueba["Anualidad"].nunique(),
    currentvalue={"prefix":"Año: "}, #el prefijo que aparece entre la grafica y justo encima del slider
    pad={"t": dfPrueba["Anualidad"].nunique()}, #el numero de steps que tendra el slider
    steps=steps,
    xanchor='left',
    y= -0.2
)]

##
#añado los sliders y estilizo un poco el grafico 
figFinancieraAnualBarrasTramitadasAsesoradas.update_layout(
    sliders=sliders,
    xaxis_title = "Periodo",
    yaxis_title = "Índice de Precios",
    legend_title="Precios",
    font=dict(
        color="black"
    ),
    title={
        'text': "Gráfica Solicitudes Tramitadas Asesoradas Anual en Barra",
        'y': 0.9,
        'x':0.4,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    
)

figFinancieraAnualBarrasTramitadasAsesoradas.update_xaxes(title_standoff=30)
##
# Edit slider labels
figFinancieraAnualBarrasTramitadasAsesoradas['layout']['sliders'][0]['currentvalue']['prefix']='Año: '
for i, date in enumerate(dfPrueba["Anualidad"].unique().tolist(), start = 0):
#for i,date in enumerate(lista, start = 0):
    figFinancieraAnualBarrasTramitadasAsesoradas['layout']['sliders'][0]['steps'][i]['label']= date
    

figFinancieraAnualBarrasTramitadasAsesoradas.show()

In [ ]:
figFinancieraAnualBarrasTramitadasOtras = go.Figure(layout_yaxis_range=[0,100])

# Add traces, one for each slider step
for years in dfPrueba["Anualidad"].unique(): #recorro un bucle con cada año
    aux = dfPrueba[dfPrueba["Anualidad"]==years] #Creo un aux que sera el df con el año que necesito
    aux1 = aux[aux["Tipo de Solicitud"] == "Solicitudes tramitadas - otras"]
    aux1 = aux1.sort_values(by=['numeroMes'])
    #hacer un bucle for
    
    for i in range(18):
            
        nombre = switch (i)
        columna = aux1[nombre]
        
        figFinancieraAnualBarrasTramitadasOtras.add_bar(
            visible=False, #las ocultamos
            x=aux1.Mes,
            name= nombre,
            y=columna
        )
        
    
# con Este bucle anidado mostramos tantas columnas como vueltas demos de la primera grafica
for i in range (18):
    figFinancieraAnualBarrasTramitadasOtras.data[i].visible = True
# Create and add slider
steps = []
#steps, son los pasos , los cambios que hay entre cada año
# es en los steps donde hacemos visibles las barras y actualizamos la grafica
for i in range(0,len(figFinancieraAnualBarrasTramitadasOtras.data),18): # recorro un bucle con todos los datos de la grafica, hago una iteracion de 2 en 2 debido a que me interesa sacar dos datos cada vez, dos barras
    step = dict(
        method="update", 
        args=[{"visible": [False] * len(figFinancieraAnualBarrasTramitadasOtras.data)},  
              {"title": "Gráfica Solicitudes Tramitadas Otras Categorías año: " + str(dfPrueba.Anualidad.unique().tolist()[int(i/18)])},
             
             ] # layout attribute y nombre de la grafica
    )
    
    for u in range(18): # Una forma optimizada y mas bonita que antes, un bucle anidado para cada barra
        step["args"][0]["visible"][i+u] = True
    
    
    steps.append(step)
#sliders
sliders = [dict(
    active=dfPrueba["Anualidad"].nunique(),
    currentvalue={"prefix":"Año: "}, #el prefijo que aparece entre la grafica y justo encima del slider
    pad={"t": dfPrueba["Anualidad"].nunique()}, #el numero de steps que tendra el slider
    steps=steps,
    xanchor='left',
    y= -0.2
)]

##
#añado los sliders y estilizo un poco el grafico 
figFinancieraAnualBarrasTramitadasOtras.update_layout(
    sliders=sliders,
    xaxis_title = "Periodo",
    yaxis_title = "Índice de Precios",
    legend_title="Precios",
    font=dict(
        color="black"
    ),
    title={
        'text': "Gráfica Solicitudes Tramitadas Otras Categorías Anual en Barra",
        'y': 0.9,
        'x':0.4,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    
)

figFinancieraAnualBarrasTramitadasOtras.update_xaxes(title_standoff=30)
##
# Edit slider labels
figFinancieraAnualBarrasTramitadasOtras['layout']['sliders'][0]['currentvalue']['prefix']='Año: '
for i, date in enumerate(dfPrueba["Anualidad"].unique().tolist(), start = 0):
#for i,date in enumerate(lista, start = 0):
    figFinancieraAnualBarrasTramitadasOtras['layout']['sliders'][0]['steps'][i]['label']= date
    

figFinancieraAnualBarrasTramitadasOtras.show()

In [ ]:
figFinancieraAnualBarras.write_html("C:/Users/david/OneDrive/Imágenes/GraficosPlotly/Economia/PlataformaFinanciera/FiguraFinancieraAnualBarras.html")
figFinancieraAnualBarrasTramitadasFinanciadas.write_html("C:/Users/david/OneDrive/Imágenes/GraficosPlotly/Economia/PlataformaFinanciera/FiguraFinancieraAnualBarrasTramitadasFinanciadas.html")
figFinancieraAnualBarrasTramitadasAsesoradas.write_html("C:/Users/david/OneDrive/Imágenes/GraficosPlotly/Economia/PlataformaFinanciera/FiguraFinancieraAnualBarrasTramitadasAsesoradas.html")
figFinancieraAnualBarrasTramitadasOtras.write_html("C:/Users/david/OneDrive/Imágenes/GraficosPlotly/Economia/PlataformaFinanciera/FiguraFinancieraAnualBarrasTramitadasOtras.html")